In [1]:
import copy

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
import pandas as pd
from tqdm import tqdm
import numpy as np

import hopsworks

device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = 'cpu'

# %load_ext dotenv
# %dotenv

In [2]:
project = hopsworks.login(project='id2223_enric')
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/197783
Connected. Call `.close()` to terminate connection gracefully.


In [3]:
hackernews_fg = fs.get_feature_group("hackernews_fg", 1)
query = hackernews_fg.select_all()
feature_view = fs.get_or_create_feature_view(name="hackernews_fv",
                                  version=1,
                                  description="Hackernews feature view",
                                  labels=["score"],
                                  query=query)

In [4]:
feature_view_df = feature_view.train_test_split(test_size=0.05)
df = pd.DataFrame(feature_view_df[0]).join(pd.DataFrame(feature_view_df[2]['score']))
df = df.sample(frac=0.50).reset_index(drop=True)
df = df.fillna(value=" ")


W20240107 12:49:48.936477 133555712 status.cc:137] DoAction result was not fully consumed: Cancelled: Flight cancelled call, with message: CANCELLED. Detail: Cancelled


Finished: Reading data from Hopsworks, using ArrowFlight (7.33s) 


In [5]:
num_samples = 1000
bins = np.linspace(df['score'].min(), df['score'].max(), num_samples + 1)
df['score_bin'] = pd.cut(df['score'], bins=bins, labels=False, include_lowest=True)
df_sampled = df.groupby('score_bin', group_keys=False).apply(lambda x: x.sample(1))
df_sampled = df_sampled.drop(columns=['score_bin'])
df = df_sampled

In [6]:
from feature_processing import load_text_encoder, to_embedding

In [7]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.ll1 = nn.Linear(768, 1024)
        self.bn1 = nn.BatchNorm1d(2)
        self.elu1 = nn.ELU()
        self.ll2 = nn.Linear(1024, 512)
        self.bn2 = nn.BatchNorm1d(2)
        self.elu2 = nn.ELU()
        self.llf = nn.Linear(512, 1)
        
    def forward(self, x):
        x = self.elu1(self.bn1(self.ll1(x)))
        x = self.elu2(self.bn2(self.ll2(x)))
        x = torch.sum(x, dim=1)
        x = self.llf(x)
        return x

In [8]:
model_1 = Model().to(device)

In [9]:
model_2 = copy.deepcopy(model_1)

In [10]:
def rearray(arr_str):
    arr_str = arr_str.strip("'").replace('\n', '').replace('[', '').replace(']', '').split()
    numpy_array = np.array(arr_str, dtype=float)
    return numpy_array

In [11]:
import re

def extract_words_from_link(link):
    # Match alphanumeric sequences
    url_str = ""
    words = re.findall(r'\b\w+\b', link)
    remove_list = ['https', 'http', 'www']
    final_words = [w for w in words if not(w in remove_list)]
    for w in final_words:
        url_str += w + " "
    return url_str

In [12]:
# class DfDataset(Dataset):
#     def __init__(self, df, col):
#         self.df = df
#         self.col = col
    
#     def __len__(self):
#         return len(self.df)
    
#     def __getitem__(self, idx):
#         val = self.df[self.col].iloc[idx]
#         reg_lbl = self.df['score'].iloc[idx]
#         if reg_lbl <= 1:
#             cls_lbl = 0
#             reg_lbl = reg_lbl
#         else:
#             cls_lbl = 1
#             reg_lbl = reg_lbl / 2800
#         arr = rearray(val)
#         return arr, cls_lbl, reg_lbl
    
from feature_processing import to_embedding
class DfDataset(Dataset):
    def __init__(self, df):
        self.df = df[['title', 'url', 'score']]
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        title = [self.df['title'].iloc[idx]]
        url = [extract_words_from_link(self.df['url'].iloc[idx])]
        score = self.df['score'].iloc[idx]/280
        
        title_embedding = F.softmax(to_embedding(title), dim=1)
        url_embedding = F.softmax(to_embedding(url), dim=1)
        
        embeddings = torch.cat([title_embedding, url_embedding], dim=0)
        embeddings = F.softmax(embeddings, dim=0)
        return embeddings, score
        

In [13]:
train_df, val_df = train_test_split(df, test_size=0.1)

In [14]:
train_ds, val_ds = DfDataset(train_df), DfDataset(val_df)

In [15]:
train_loader = DataLoader(train_ds, batch_size=12, 
                          num_workers=2, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=12,
                        num_workers=2, shuffle=True)

In [16]:
epochs = 500
optimizer = optim.AdamW(model_1.parameters(), lr=1e-5)
lr_scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.01, epochs=500, steps_per_epoch=len(train_loader))
mse_loss = nn.MSELoss()
bce_loss = nn.BCELoss()
def loss_fn(output, Y):
    cls_lbl, reg_lbl = Y[0], Y[1]
    cls_op, reg_op = F.sigmoid(output[:, 0]), output[:, 1]
    bce_l = bce_loss(cls_op, cls_lbl)
    mse_l = mse_loss(reg_op * cls_lbl, reg_lbl *  cls_lbl)
    return bce_l + mse_l

In [21]:
for epoch in range(epochs):
    tr_loss_per_batch = []
    val_loss_per_batch = []
    for sample in tqdm(train_loader):
        X, Y = sample
        X, Y = X.to(torch.float32).to(device), Y.to(torch.float32).to(device)
        target = model_1(X)
        loss = mse_loss(target.squeeze(), Y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        tr_loss_per_batch.append(loss.item())
        lr_scheduler.step()
    with torch.no_grad():
        for sample in tqdm(val_loader):
            X, Y = sample
            X, Y = X.to(torch.float32).to(device), Y.to(torch.float32).to(device)
            target = model_1(X)
            loss = mse_loss(target, Y)
            val_loss_per_batch.append(loss.item())
            
    print(f"Epoch: {epoch+1}/{epochs}")
    print(f"Training loss: {np.mean(tr_loss_per_batch)} Validation Loss: {np.mean(val_loss_per_batch)}")

  0%|          | 0/4990 [00:00<?, ?it/s]Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/enric/anaconda3/lib/python3.11/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/enric/anaconda3/lib/python3.11/multiprocessing/spawn.py", line 132, in _main
    self = reduction.pickle.load(from_parent)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute 'DfDataset' on <module '__main__' (built-in)>
  0%|          | 0/4990 [00:04<?, ?it/s]


KeyboardInterrupt: 

In [17]:
torch.save(model_1,'model.pth')

In [18]:
model_1 = torch.load('model.pth', map_location=torch.device('cpu'))

In [19]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

In [20]:
X, Y = train_loader.dataset[0]
X_df = pd.DataFrame(X.numpy())
input_schema = Schema(X_df)
Y_df = pd.DataFrame([Y])
output_schema = Schema(Y_df)
model_schema = ModelSchema(input_schema, output_schema)

In [24]:
mr = project.get_model_registry()
hopsworks_model = mr.python.create_model(
        name="hackernews_model",
        model_schema=model_schema,
        description="Hackernews upvote predictor"
    )

Connected. Call `.close()` to terminate connection gracefully.


In [25]:
import pickle
pickle.dump(model_1, open("model.pkl", "wb"))

In [26]:
hopsworks_model.save("model.pkl")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/6 [00:00<?, ?it/s]

Model created, explore it at https://c.app.hopsworks.ai:443/p/197783/models/hackernews_model/3


Model(name: 'hackernews_model', version: 3)